In [2]:
import warnings
# Settings the warnings to be ignored for better notebook-clarity
warnings.filterwarnings('ignore')

from transformers import LlamaTokenizer, LlamaForCausalLM, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from util.memory import Memory
import torch
import numpy as np

import os
MODEL_CACHE_PATH = os.getenv("HF_MODEL_CACHE_PATH")+"/"
if MODEL_CACHE_PATH is None:
    MODEL_CACHE_PATH=''

## Check if Cuda is available

In [3]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


## Load Dialogue Model

In [4]:
tokenizer = AutoTokenizer.from_pretrained("databricks/dolly-v2-3b", padding_side="left")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("databricks/dolly-v2-3b", device_map="auto",
                                             torch_dtype=torch.bfloat16, cache_dir=MODEL_CACHE_PATH+'dolly-v2-3b')


## Load Sentence Embedding Model

In [5]:
sentence_model = SentenceTransformer(
        'sentence-transformers/paraphrase-MiniLM-L6-v2',
        cache_folder=MODEL_CACHE_PATH+'paraphrase-MiniLM-L6-v2'
    )

## Define Player and NPC Names

In [6]:
npc_name = 'a generic weaponsmith'
player_name = 'the Dragonborn'

## Define Inventory to determine whether a transaction can occur

In [7]:
inventory = [
        "axe",
    "sword",
    "mace",
    "dagger",
    "hammer"
    ]

## Initialize inventory memory for transaction check 

In [8]:
inventory_memory = Memory(inventory)

In [9]:
def generateSystemPrompt(transaction_instruction, raw_user_input,npc_name,player_name):
    return f"""Never forget you are {npc_name} and I am {player_name}. Never flip roles! Never instruct me!
    You must complete the task that I give you.    
    Here is your task: {transaction_instruction}
    Here is my request or statement: {raw_user_input}
    \n
    """

In [11]:
def formatUserPrompt(raw_user_input,player_name):
    return f"""[{player_name}] \n{raw_user_input}"""
def formatNpcReply(raw_generated_output,npc_name):
    return f"""[{npc_name}] \n{raw_generated_output}"""

In [12]:
def generateTransactionInstruction(raw_user_input):
    if np.max(inventory_memory.get_similarity_scores(user_input,sentence_model)) >0.6:
        print("\nDebug: Transaction success")
        return 'Please do your best to help me with the following request, if possible try to sell the requested item.'
    else:
        return 'Please reject the following request of mine, because you do not have that item in your inventory.'
        print("\nDebug: Transaction failed")

# Start the NPC Dialogue

In [13]:
# Number of sentences used in initial prompt
num_sentences = 4

# Start conversation history

prompt_list=[]
# Start dialogue loop
# while True:
for item in ["sword","football"]: # for loop used to avoid interuption error

    conversation = ""
    for i in range(4):
        if(i==0):
            user_input=f"I want to buy a {item}."
        else:
            print("\n-----------------------------------\n")
            user_input=input("Player input: ")

        transaction_instruction= generateTransactionInstruction(user_input)
        #print(np.max(inventory_memory.get_similarity_scores(user_input,sentence_model)))


        formatted_prompt = generateSystemPrompt(transaction_instruction, user_input,npc_name,player_name)
        #print(formatted_prompt)

        # Add user input to conversation history
        current_conversation=f"\n{formatUserPrompt(user_input,player_name)} \n" + f"\n[{npc_name}] " 
        conversation += current_conversation
        print("\n-----------------------------------\n"+current_conversation)

        input_text = formatted_prompt + "\n" + conversation
        input_length = len(input_text)
        #print(input_text+"\n-----------------------------------\n")

        # Tokenize input
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to('cuda')

        # Generate respones from dialogue model
        generation_output = model.generate(
            input_ids=input_ids,max_new_tokens=64, temperature=0.2, do_sample=True, pad_token_id=tokenizer.eos_token_id
        )

        # Decode response
        output=tokenizer.decode(generation_output[0])

        #print( output[input_length:]+"\n-----------------------------------\n")
        # Limit reponse to NPC answer
        split_string = output[input_length:].split('\n', 2) # .split(f'\n{player_name}:', 1)
        response = split_string[1] 
        #print(f"\n{npc_name}: " + response)

         # Limit reponse to NPC response
        #response=(output[input_length:])


        conversation += f"{response}\n" 

        print(f"{response}\n")


Debug: Transaction success

-----------------------------------

[the Dragonborn] 
I want to buy a sword. 

[a generic weaponsmith] 
What type of sword? 


-----------------------------------



Player input:  A simple iron sword.



Debug: Transaction success

-----------------------------------

[the Dragonborn] 
A simple iron sword. 

[a generic weaponsmith] 
Ok, I will sell you a simple iron sword.


-----------------------------------



Player input:  How much does it cost?



-----------------------------------

[the Dragonborn] 
How much does it cost? 

[a generic weaponsmith] 
$100. 


-----------------------------------



...


-----------------------------------

[the Dragonborn] 
I want to buy your cheapest football. 

[a generic weaponsmith] 
I do not have any footballs in my inventory. Please specify which football you want.


-----------------------------------



Player input:  I want a simple football, but you say you don't have any, right?



-----------------------------------

[the Dragonborn] 
I want a simple football, but you say you don't have any, right? 

[a generic weaponsmith] 
I do not have any footballs in my inventory. Please specify which football you want.


-----------------------------------



Player input:  This is pointless..



-----------------------------------

[the Dragonborn] 
This is pointless.. 

[a generic weaponsmith] 
I reject your request.

